In [2]:
# Import librairies ----
import numpy as np
import pandas as pd

## RDKit
import rdkit 
from rdkit import Chem
from rdkit.Chem import AllChem

## Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

In [4]:
# Import data ----
train = pd.read_csv('data/x_train.csv')
test = pd.read_csv('data/x_test.csv')

y_train = pd.read_csv('data/y_train.csv')['y']

print(f"Le dataset d'entraînement contient {train.shape[0]} molécules.")
print(f"le dataset de test contient {test.shape[0]} molécules.")

train.head()

Le dataset d'entraînement contient 4400 molécules.
le dataset de test contient 2934 molécules.


,id,smiles
0,0,CNC(=O)c1ccccc1Sc1ccc2c(C#Cc3cccc(NCCOC)c3)n[n...
1,1,CC(C)(C)c1ccc(Nc2nnc(-c3cnccc3CCc3ccncc3)o2)cc1
2,2,CN1CCN(CCCn2ccc(-c3cnc4c(-c5ccsc5)cnn4c3)cc2=O...
3,3,CCn1c2ccc(NC(=O)Nc3ccc(OC)cc3)cc2c2c3c(c4c(c21...
4,4,Cc1ccc(C(=O)Nc2cccc(C(C)C)c2)cc1N1CCc2ncncc2C1
